# Stacking Ensembles

In [1]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train['Sex_binario'] = train['Sex'].map({"male": 0, "female": 1})
test['Sex_binario'] = test['Sex'].map({"male": 0, "female": 1})

In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_binario
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


# Feature Selection

_select.dtypes_ seleciona todas as colunas que contenham o tipo de dado especificado, neste caso __np.number__ <br>
Esta maneira simplificada de seleção de features é apenas para que possamos entender a tecnica de _Stacking_ , não necessidade de muito trabalho nesta parte pois o objetivo não é ganhar a competição.

In [6]:
X = train.select_dtypes(include=np.number).drop(["PassengerId", 'Survived'], axis=1).fillna(0)
y = train['Survived']

# Modelagem

In [7]:
#Libs
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.pipeline import make_pipeline

## Validação cruzada

In [8]:
kf = KFold(n_splits=2, random_state=0, shuffle=True)

#Dois ciclos de treinos e validação!
"""
Primeiro ciclo:
    Treino com a 'PRIMEIRA METADE' dos dados e validação com 'SEGUNDA METADE' dos dados
    
Segundo ciclo:
    Treino com a 'SEGUNDA METADE' dos dados e validação com 'PRIMEIRA METADE' dos dados
    
OBS: Tal metódo se chama Reamostragem - normalmente é feito com mais divisões do dataset (5 ou até 10)
"""

#Criando uma matriz de zeros para armazenar futuramente as previsões. Tal matriz tem o nº linhas do X e 4 colunas,
#pois serão usados 4 modelos neste stacking
second_level = np.zeros((X.shape[0], 4))

for tr, ts in kf.split(X,y):
    Xtr, Xval = X.iloc[tr], X.iloc[ts]
    ytr, yval = y.iloc[tr], y.iloc[ts]
    
    rf = RandomForestClassifier(n_estimators=100, n_jobs=6, random_state=10)
    rf.fit(Xtr, ytr)
    prf = rf.predict_proba(Xval)[:,1]
    prf_ = (prf > 0.5).astype(int) #.5 é o ponto de corte padrão para acurácia, adotado pelo sklearn
    print("RF Accuracy: {} - Log Loss: {}".format(accuracy_score(yval, prf_), log_loss(yval, prf)))
    
    et = ExtraTreesClassifier(n_estimators=100, n_jobs=6, random_state=10)
    et.fit(Xtr, ytr)
    pet = et.predict_proba(Xval)[:,1]
    pet_ = (pet > 0.5).astype(int)
    print("ET Accuracy: {} - Log Loss: {}".format(accuracy_score(yval, pet_), log_loss(yval, pet)))
    
    #Padronização dos dados com o modelo mais comum - StandardScaler - basicamente padroniza os dados pelo desvio
    #padrão
    lr1 = make_pipeline(StandardScaler(), LogisticRegression())
    lr1.fit(Xtr, ytr)
    plr1 = lr1.predict_proba(Xval)[:,1]
    plr1_ = (plr1 > 0.5).astype(int)
    print("LR StdScaler Accuracy: {} - Log Loss: {}".format(accuracy_score(yval, plr1_), log_loss(yval, plr1)))
    
    #Padronização dos dados com o modelo MinMaxScaler que basicamente padroniza os dados com os seus valores min e
    #max
    lr2 = make_pipeline(MinMaxScaler(), LogisticRegression())
    lr2.fit(Xtr, ytr)
    plr2 = lr2.predict_proba(Xval)[:,1]
    plr2_ = (plr2 > 0.5).astype(int)
    print("LR MinMax Accuracy: {} - Log Loss: {}".format(accuracy_score(yval, plr2_), log_loss(yval, plr2)))
    
    second_level[ts, 0] = prf
    second_level[ts, 1] = pet
    second_level[ts, 2] = plr1
    second_level[ts, 3] = plr2
    
    print()
    
    

RF Accuracy: 0.7869955156950673 - Log Loss: 0.8891877224352214
ET Accuracy: 0.7802690582959642 - Log Loss: 1.7277767849421113
LR StdScaler Accuracy: 0.7713004484304933 - Log Loss: 0.4548870233895985
LR MinMax Accuracy: 0.773542600896861 - Log Loss: 0.45730271249069515

RF Accuracy: 0.8292134831460675 - Log Loss: 0.6313360340802087
ET Accuracy: 0.8089887640449438 - Log Loss: 2.533916920090966
LR StdScaler Accuracy: 0.7955056179775281 - Log Loss: 0.4579224442106097
LR MinMax Accuracy: 0.802247191011236 - Log Loss: 0.4610789905245172



__OBS:__ _Vemos que a aleatoriadade dos dados, implicada pelo KFold, altera drasticamente o desempenho dos modelos_

__Accuracy:__ Taxa de acerto, dos valores previstos pelo modelo quantos são iguais ao y_verdadeiro do conjunto de treino. <br>
__Log Loss:__ Calibração do modelo, A medida que a probabilidade da previsão aproxima-se de 1.0 a perda (log) diminui, porem para probabilidades menores que 1.0 a perda(log) aumenta rapidamente, ou seja, esta metrica penaliza os dois tipos de erros, mas penaliza especialmente aquelas previões "muito erradas"

## Stacking

Esta segunda parte, usaremos a matriz de previsões criadas anteriormente (__second_level__) como features e o mesmo __y__, para treinarmos o modelo

In [15]:
#Modelo de uma base line possível para o ensemble:
# A média das colunas da matriz second_level (que são as previsões feitas na primeira etapa)

#second_level.mean(axis=1)

In [16]:
for tr, ts in kf.split(X,y):
    
    Xtr, Xval = second_level[tr], second_level[ts]
    ytr, yval = y.iloc[tr], y.iloc[ts]
    
    lr_stack = LogisticRegression(C=1.)
    lr_stack.fit(Xtr, ytr)
    plr_stack = lr_stack.predict_proba(Xval)[:,1]
    plr_stack_ = (plr_stack > 0.5).astype(int)
    
    print("Stack Accuracy: {} - Log Loss: {}".format(accuracy_score(yval, plr_stack_), log_loss(yval, plr_stack)))
    print()

Stack Accuracy: 0.8004484304932735 - Log Loss: 0.43791529135624363

Stack Accuracy: 0.8157303370786517 - Log Loss: 0.4289260815847779



__Devido a aleatoriade o resultado do Ensemble pode ou não bater os modelos individuais, tudo depende da aleatoriadade, rodando diversas vezes, o ensemble ganhará em alguns e outras não.__ <br>
__Notando que a log loss diminui pois se beneficia deste processo__

___DICA: Para um ensemble, é bom ter modelos que cometem erros diferentes entre si, ou seja, os modelos performam de maneiras diferentes dependendo da "seção" do dataset___

### Por que utilizar o KFold as duas vezes???

In [17]:
""""
folds = [1,2,3]

#[features_fold_i, previsões_fold_j][linhas_que_geraram_as_previsões_k]

Primeiro Nível
ciclo_1 = [1,2] [3]
ciclo_2 = [1,3] [2]
ciclo_1 = [2,3] [1]

Segundo Nível
ciclo_1 = [1,2] [3]
ciclo_2 = [1,3] [2]
ciclo_1 = [2,3] [1]
#Neste cenário (igual ao primeiro ciclo) o modelo ja conhece alguns dos dados e isso além de redundante, que pode
#ser prejudicial para o modelo, acarreta em vazamento de dados

"""

'"\nfolds = [1,2,3]\n\n#[features_fold_i, previsões_fold_j][linhas_que_geraram_as_previsões_k]\n\nPrimeiro Nível\nciclo_1 = [1,2] [3]\nciclo_2 = [1,3] [2]\nciclo_1 = [2,3] [1]\n\nSegundo Nível\nciclo_1 = [1,2] [3]\nciclo_2 = [1,3] [2]\nciclo_1 = [2,3] [1]\n#Neste cenário (igual ao primeiro ciclo) o modelo ja conhece alguns dos dados e isso além de redundante, que pode\n#ser prejudicial para o modelo, acarreta em vazamento de dados\n\n'

Como a ideia é, através da validação cruzada, fazer com que os dados se dividam aleatóriamente e assim possamos utlizar nos ciclos, uma parte para treino e outra para validação, o KFold garante que nao haverá vazamento de dados entre os ciclos como por exemplo: no segundo ciclo treinar o modelo com o conjunto de treino do primeiro ciclo! 

# Correlações entre as previsões do modelo

In [18]:
pd.DataFrame(np.corrcoef(second_level.T))

,0,1,2,3
0,1.000000,0.966815,0.808323,0.808835
1,0.966815,1.000000,0.756249,0.756569
2,0.808323,0.756249,1.000000,0.995999
3,0.808835,0.756569,0.995999,1.000000


'0' -> RandomForest <br>
'1' -> ExtraTree <br>
'2' -> LogisticRegression with StandardScaler <br>
'3' -> LogisticRegression with MinMaxScaler <br>

__Para o Ensemble é melhor quando os modelos possuem previsões pouco correlacionadas, ou seja, erram em pontos e por motivos diferentes__